In [ ]:
from elasticsearch_dsl import connections, Index, Search
from elasticsearch_dsl.query import Range, Terms, Term
from elasticsearch_dsl.aggs import Terms as PerTerms, Cardinality
from json import loads, load
from pandas import DataFrame, NA
from pathlib import Path
from requests import get

In [ ]:
ELASTIC_HOST = "https://elasticsearch.srv.webis.de"
ELASTIC_PORT = 9200
ELASTIC_USER = "po87xox"
# As a way to hide the password at least from the notebook, enter a path to a file here, which only contains the password for Elastic.
ELASTIC_PASSWORD_FILE = Path("passwords/webis-elasticsearch.txt").expanduser()
INDEX = "corpus_mastodon_statuses*"

INSTANCE_DATA_PATH = Path("../data/instance_data.jsonl")
INSTANCES_PATH = Path("../data/instances.txt")
REMOVED_INSTANCES_PATH = Path("../data/uncrawlable_instances.json")

OUTPUT_PATH = Path("../data")

NUM_TOP_INSTANCES = 10

# Limit the Elastic searches to a specific date range. Crawling started on 2023-12-21.
DATE_AFTER = "2023-12-01T00:00:00"
## Ca. "2024-01-30T12:00:00" is the time when a new version of the fediverse data was gahtered.
DATE_BEFORE = "2024-02-22T00:00:00"

In [ ]:
# Connect to Elastic.
with ELASTIC_PASSWORD_FILE.open("r") as f:
    password = f.readline().strip("\n")
elastic = connections.create_connection(
    hosts=ELASTIC_HOST + ":" + str(ELASTIC_PORT),
    basic_auth=(ELASTIC_USER, password),
    timeout=3000
)

In [ ]:
# Prepare the date query
date_query = Range(crawled_at={"gte": DATE_AFTER, "lte": DATE_BEFORE, "format" : "date_hour_minute_second"})
date_query.to_dict()

In [ ]:
base_search: Search = Index(INDEX)\
    .search()\
    .filter(date_query)

In [ ]:
instance_name = "veganism.social"
language_name = "en"

filtered_search: Search = base_search.filter(Term(instance=instance_name)).filter(Term(language=language_name))

In [19]:
from detoxify import Detoxify

toxic_bert = Detoxify('original')

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /home/julian/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
response = filtered_search.extra(size=10).execute()

In [16]:
class Toxicity:
    def __init__(self, toxicity=0, severe_toxicity=0, obscene=0, threat=0, insult=0, identiy_attack=0):
        self.toxicity = toxicity
        self.severe_toxicity = severe_toxicity
        self.obscene = obscene
        self.threat = threat
        self.insult = insult
        self.identity_attack = identiy_attack
    
    def __add__(self, other):
        return Toxicity(
            self.toxicity + other.toxicity,
            self.severe_toxicity + other.severe_toxicity,
            self.obscene + other.obscene,
            self.threat + other.threat,
            self.insult + other.insult,
            self.identity_attack + other.identity_attack
        )

In [17]:
# size = filtered_search.count()
toxicity_sum = Toxicity()

for hit in response: # filtered_search.scan():
    toxic_values = toxic_bert(hit.content)
    toxicity_sum += Toxicity(toxic_values)


NameError: name 'toxic_bert' is not defined

In [ ]:
toxicity_sum